# Demo: run Monte Carlo EMs on simulated hypergraphs

There are four variants of MCEM available for hypergraphs.
- 1: 'rbst-prcr'
- 2: 'rbst-norm'
- 3: 'aggr-prcr'
- 4: 'aggr-norm'

MCEM algorithm

- Input
    - uniform/general hyperedge sets

- Output 
    - shape parameter $\hat{\beta}$
    - feature covariate $\hat{X}$


In [1]:
suppressPackageStartupMessages({
    library(Rcpp)
    library(geometry) # calculate convex hull volume
    library(MASS)
    library(mnormt) # generate mulvariate gaussian in porposal
    library(ggplot2)
    library(igraph) # graph alg and visualization
    library(CVXR) # optimization used in warm start in mcem
})
sourceCpp('../code/helper_mcem.cpp') # mcem algorithms
sourceCpp('../code/helper_comb.cpp') # combinatorial algorithms
source('../code/helper_general.R') # general helpers
source('../code/helper_mcem_cpp_user.R') # MCEM algorithms R wrapper using mcem helpers in cpp
source('../code/helper_mcem_general_hg.R')

# 1. Generate uniform/general hypergraphs

In [2]:
n.demo = 7                                              #--------Model specs-----#
if (TRUE){ # uniform hypergraph
    mix.k.demo = c(2)                                   #--------Model specs-----#
    mix.true.beta.demo =c(1e-1)                         #--------Model specs-----#
}
if (FALSE){ # general hypergraph
    mix.k.demo = c(2,3)                                 #--------Model specs-----#
    mix.true.beta.demo =c(1e-1,1e-3)                    #--------Model specs-----#
}
mix.edge.demo = mix.count.demo = mix.subset.mat.demo = list()
for (i in 1:len(mix.k.demo)){
        #i=1
        k.demo = mix.k.demo[i]
        true.beta.demo = mix.true.beta.demo[i]
        num.sample.demo  = 10*choose(n.demo, k.demo)
        p.demo = 2
        mu.demo = 0
        var.demo = 1
        feat.mat.demo = get.gaussian.featmat(nrow = n.demo, ncol = p.demo,
                                            mean = mu.demo, var1 = var.demo, cov12 = 0)
        feat.mat.demo = standerize.matrix(feat.mat.demo)
        dim(feat.mat.demo)
        save(feat.mat.demo, file = 'feat.mat.demo.standarized.n50p2.Rdata')
        mix.subset.mat.demo[[i]] = find_subset(n.demo, k.demo)
        shape.vec = c('gamma','pareto','gaussian', 'window')
        which.shape.demo = shape.vec[1]
        save.path = "./output/"
        sim.data =sample.from.shape.cvxhull(n = n.demo, k = k.demo,
                                            subset.mat = mix.subset.mat.demo[[i]],
                                            feat.mat = feat.mat.demo,
                                            num.sample = num.sample.demo,
                                            which.shape = which.shape.demo, param.vec = c(1, true.beta.demo),
                                            if.save.sample = F, save.path = save.path )
        mix.edge.demo[[i]] = sim.data$edge.sampled
        mix.count.demo[[i]] = sim.data$count.vec
}



# Run MCEM

MCEMs dict:
- 1: 'rbst-prcr'
- 2: 'rbst-norm'
- 3: 'aggr-prcr'
- 4: 'aggr-norm'


In [3]:
set.seed(123)

# Pick one mcem from the list
which.mcem.demo = 1
res=run.mcem.general.hypergraph(which.mcem=which.mcem.demo,isVerboseResult = FALSE, isSaveResult = FALSE)

print('hypergraph cardinalities:')
print(mix.k.demo)
print('beta est are:')
print(res$beta.hat)
print('feat.est is:')
print(res$feat.hat)

[1] "rbst-prcr" "is chose."
Aug-16-06-57-55 | #### check mcem is to run on k = 2.00
Aug-16-06-57-55 | MCEM on  k = 2 
[1] "MCEM: robust-prcr"
Aug-16-06-57-56 |  *** robust MCEM on 2-unif hg is starting 
Aug-16-06-57-56 | Initial round: lkhd = -6.39350e+02, beta  = 0.000 
Aug-16-06-57-56 | 2th round: lkhd.estep = -6.39350e+02, beta = 0.113, lkhd.mstep = -6.21958e+02 
Aug-16-06-57-56 | 3th round: lkhd.estep = -6.30375e+02, beta = 0.089, lkhd.mstep = -6.29686e+02 
Aug-16-06-57-56 | 4th round: lkhd.estep = -6.26770e+02, beta = 0.117, lkhd.mstep = -6.25627e+02 
Aug-16-06-57-56 | 5th round: lkhd.estep = -6.37482e+02, beta = 0.065, lkhd.mstep = -6.32619e+02 
Aug-16-06-57-56 | 6th round: lkhd.estep = -6.31906e+02, beta = 0.080, lkhd.mstep = -6.30479e+02 
Aug-16-06-57-56 | 7th round: lkhd.estep = -6.36560e+02, beta = 0.056, lkhd.mstep = -6.34108e+02 
Aug-16-06-57-56 | 8th round: lkhd.estep = -6.38935e+02, beta = 0.046, lkhd.mstep = -6.35990e+02 
Aug-16-06-57-56 | 9th round: lkhd.estep = -6.3572